In [19]:
!git clone https://github.com/Namanj1298/Training-Data-Track-1

fatal: destination path 'Training-Data-Track-1' already exists and is not an empty directory.


In [20]:
ls Training-Data-Track-1

 Volume in drive C is Windows
 Volume Serial Number is 6E73-904C

 Directory of C:\Users\hp\Desktop\Machine Learning\Udemy Course\Training-Data-Track-1

10-09-2020  21:51    <DIR>          .
10-09-2020  21:51    <DIR>          ..
10-09-2020  21:43                68 .gitattributes
07-09-2020  08:28         2,273,145 driving_log.csv
10-09-2020  21:51    <DIR>          IMG
               2 File(s)      2,273,213 bytes
               3 Dir(s)  38,537,932,800 bytes free


In [22]:
import os
import numpy as np
import keras 
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Convolution2D
import ntpath
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import cv2

In [23]:
import matplotlib.pyplot as plt
import matplotlib.image as mping
%matplotlib inline

AttributeError: module 'matplotlib' has no attribute 'get_data_path'

In [16]:
datadir = 'Training-Data-Track-1'
columns = ['center','left','right','steering','throttle','reverse','speed']
data = pd.read_csv(os.path.join(datadir, 'driving_log.csv'), names = columns)
pd.set_option('display.max_colwidth', None)
data.head()

NameError: name 'pd' is not defined

In [ ]:
def path_leaf(path):
  head,tail = ntpath.split(path)
  return tail
data['center'] = data['center'].apply(path_leaf)
data['left'] = data['left'].apply(path_leaf)
data['right'] = data['right'].apply(path_leaf)
data.head()

In [ ]:
num_bins = 25
samples_per_bin=250
hist, bins = np.histogram(data['steering'], num_bins)
center = (bins[:-1]+bins[1:]) * 0.5
plt.bar(center, hist, width = 0.05)
plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin, samples_per_bin))

In [ ]:
print('Total:', len(data))

remove_list = []
for j in range(num_bins):
  list_ = []
  for i in range(len(data['steering'])):
    if data['steering'][i] >= bins[j] and data['steering'][i] <= bins[j+1]:
      list_.append(i)
  list_= shuffle(list_)
  list_ = list_[samples_per_bin:]
  remove_list.extend(list_)  

print('Removed', len(remove_list))
data.drop(data.index[remove_list], inplace = True)
print('Remaining', len(data))

hist, _ = np.histogram(data['steering'], (num_bins))
plt.bar(center, hist, width=0.05)
plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin, samples_per_bin ))



In [ ]:
print(data.iloc[1])
def load_img_steering(datadir, df):
  image_path = []
  steering = []
  for i in range(len(data)):
    indexed_data = data.iloc[i]
    center, left, right  = indexed_data[0], indexed_data[1], indexed_data[2]
    image_path.append(os.path.join(datadir, center.strip()))
    steering.append(float(indexed_data[3]))
  image_paths = np.asarray(image_path)
  steerings = np.asarray(steering)
  return image_paths, steerings

image_paths, steerings = load_img_steering(datadir + '/IMG', data)   

In [ ]:
X_train, X_valid, y_train, y_valid  = train_test_split(image_paths, steerings, test_size = 0.2, random_state = 6)
print('Training Samples:  {}\n Valid SamplesL:  {}' .format(len(X_train), len(X_valid)))

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,4))
axes[0].hist(y_train, bins = num_bins, width = 0.05, color = 'blue')
axes[0].set_title('Training Set')
axes[1].hist(y_valid, bins = num_bins, width = 0.05, color = 'red')
axes[1].set_title('Validation Set')


In [ ]:
def image_preprocess(img):
  img = mpimg.imread(img)
  img = img[60:135,:,:]
  img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
  img = cv2.GaussianBlur(img, (3,3), 0)
  img = cv2.resize(img, (200,66))
  img = img/255
  return img

In [ ]:
image = image_paths[0]
original_image = mpimg.imread(image)
img_preprocess = image_preprocess(image)
fig, axes = plt.subplots(1,2,figsize=(15,10))
fig.tight_layout()
axes[0].imshow(original_image)
axes[0].set_title('Original Image')
axes[1].imshow(img_preprocess)
axes[1].set_title('PreProcessed Image')


In [ ]:
X_train = np.array(list(map(image_preprocess, X_train)))
X_valid = np.array(list(map(image_preprocess, X_valid)))
plt.imshow(X_train[random.randint(0, len(X_train)-1)])
plt.axis('off')
print(X_train.shape)

In [ ]:
    def nvidia_model():
      model = Sequential()
      model.add(Convolution2D(24, (5, 5), (2, 2), input_shape=(66, 200, 3), activation='elu'))
      model.add(Convolution2D(36, (5, 5), (2, 2), activation='elu'))
      model.add(Convolution2D(48, (5, 5), (2, 2), activation='elu'))
      model.add(Convolution2D(64, (3, 3), activation='elu'))
      
      model.add(Convolution2D(64, (3, 3), activation='elu'))
      model.add(Dropout(0.5))
      
      
      model.add(Flatten())
      
      model.add(Dense(100, activation = 'elu'))
      model.add(Dropout(0.5))
      
      model.add(Dense(50, activation = 'elu'))
      model.add(Dropout(0.5))
      
      model.add(Dense(10, activation = 'elu'))
      model.add(Dropout(0.5))
     
      model.add(Dense(1))
      
      optimizer = Adam(lr=1e-4)
      model.compile(loss='mse', optimizer=optimizer)
      return model

In [ ]:
model = nvidia_model()
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs = 30, validation_data= (X_valid, y_valid), batch_size = 100, verbose = 1, shuffle = 1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training', 'validation'])
plt.title('Loss')
plt.xlabel('Epoch')

In [ ]:
model.save('modnj.h5')

In [ ]:
del model
from keras.models import load_model
model = load_model('modnj.h5')
print(model.summary())